<a href="https://colab.research.google.com/github/heet25itachi/TIMEPASS_BUDDY/blob/main/SmartLedgerforAiran.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import time
from typing import Dict, List, Any, Optional

# Define the structure for an immutable transaction
Transaction = Dict[str, Any]

class AiranFinstockPrivateLimitedSmartLedger:
    """Simulates a secure, stateful ledger for Airan Finstock Private Limited, mimicking key
    features of smart contract: managed state, controlled functions,
    and an immutable transaction log.
    """

    def __init__(self, firm_name: str = "Airan Finstock Private Limited"):
        """Initialize the ledger with starting cash and empty holdings."""
        self.firm_name = firm_name
        # State variables managed by the "contract"
        self._cash_balance: float = 0.0
        self._holdings: Dict[str, float] = {}   # {'TICKER': quantity}

        # Immutable log (like a blockchain ledger)
        self._transaction_log: List[Transaction] = []

        print(f"[{firm_name}] Smart Ledger Initialized.")

    # ---- External Functions (The "Contract Methods")-------

    def deposit(self, amount: float, source: str) -> bool:
        """
        Deposits cash into the firm's balance.

        Args:
          amount: The amount to deposit. Must be positive.
          source: Identifier for the deposit source (e.g., 'Initial Funding').
        """

        if amount <= 0:
            print(f"[ERROR] Deposit amount must be positive. Received: {amount}")
            return False

        self._cash_balance += amount

        # Record the transaction
        self._log_transaction("DEPOSIT", amount, source=source)

        print(f"[SUCCESS] Deposited ${amount:,.2f}. New Balance: ${self._cash_balance:,.2f}")
        return True

    def execute_trade(self,
                      ticker: str,
                      side: str,
                      quantity: float,
                      price_per_unit: float) -> bool:
        """
        Executes a trade (BUY or SELL), updating cash and holdings.
        This function enforces the contract's rules.

        Args:
            ticker: The Asset symbol (e.g., 'GOOGL', 'AAPL').
            side: 'BUY' or 'SELL'
            quantity: Number of units to trade(must be positive).
            price_per_unit: The execution price.
        """

        side = side.upper()

        if quantity <= 0 or price_per_unit <= 0:
            print("[ERROR] Quantity and price must be positive.")
            return False

        total_cost = quantity * price_per_unit

        if side == 'BUY':
            if total_cost > self._cash_balance:
                print(f"[ERROR] Insufficient funds. Need ${total_cost:,.2f}, have ${self._cash_balance:,.2f}.")
                return False

            # Execute
            self._cash_balance -= total_cost
            self._holdings[ticker] = self._holdings.get(ticker, 0.0) + quantity

            self._log_transaction(side, total_cost, ticker, quantity, price_per_unit)
            print(f"[SUCCESS] BOUGHT {quantity:,.2f} of {ticker} for ${total_cost:,.2f}.")

        elif side == 'SELL':
            current_holding = self._holdings.get(ticker, 0.0)
            if quantity > current_holding:
                print(f"[ERROR] Insufficient holding of {ticker}. Need to sell {quantity:,.2f}.")
                return False

            # Execute
            self._cash_balance += total_cost
            self._holdings[ticker] -= quantity

            # Clean up Zero balance
            if self._holdings[ticker] < 1e-9:  # Handle floating point precision near zero
                del self._holdings[ticker]

            self._log_transaction(side, total_cost, ticker, quantity, price_per_unit)
            print(f"[SUCCESS] SOLD {quantity:,.2f} of {ticker} for ${total_cost:,.2f}.")

        else:
            print(f"[ERROR] Invalid trade side: {side}. Must be 'BUY' or 'SELL'.")
            return False

        return True

    #---- Read-Only Functions (Getters) ----

    def get_cash_balance(self) -> float:
        """Returns the current liquid cash balance."""
        return self._cash_balance

    def get_holdings(self) -> Dict[str, float]:
        """Return a copy of the current asset holdings."""
        return self._holdings.copy()

    def get_transaction_log(self) -> List[Transaction]:
        """
        Returns the immutable, historical transaction log.
        This data should NEVER be modifiable externally.
        """

        return self._transaction_log.copy()    # Return a copy for immutability

    #--- Internal (Private) Contract Function ----

    def _log_transaction(self,
                         type: str,
                         amount: float,
                         ticker: Optional[str] = None,
                         quantity: Optional[float] = None,
                         price: Optional[float] = None,
                         source: Optional[str] = None):
        """ Internal function to record an immutable transaction timestamped."""
        self._transaction_log.append({
            "timestamp" : time.time(),
            "type" : type,
            "amount" : amount,
            "ticker" : ticker,
            "quantity" : quantity,
            "price": price,
            "cash_after_tx" : self._cash_balance,
            "source" : source if source else "Trading Engine"
        })

# --- Example Usage

if __name__ == '__main__':
    # Initialize the ledger
    firm_ledger = AiranFinstockPrivateLimitedSmartLedger()

    #1. Deposit Initial Capital
    firm_ledger.deposit(amount=500000.00, source = "Seed Capital")

    print("\n--- Trading Activity ---")

    #2. Execute a Buy Trade
    # Buy 100 shares of AAPL at $180.00
    firm_ledger.execute_trade(ticker="AAPL", side="BUY", quantity=100.0, price_per_unit=180.00)

    #3. Execute another Buy Trade (Different asset)
    # Buy 50 Shares of GOOGL at $1500.00 (large purchase)
    firm_ledger.execute_trade(ticker="GOOGL", side="BUY", quantity=50.0, price_per_unit=1500.00)

    #4. Attempt a trade that fails (Insufficient Funds)
    # Attempt to buy 1000 share of MSFT at $400 (Requires $400,000)
    print("\n--- Attempting to Buy Failed Trade ----")
    firm_ledger.execute_trade(ticker="MSFT", side="BUY", quantity=1000.0, price_per_unit=400.00)

    #5. Execute a Sell Trade
    # Sell 50 shares of AAPL at $195.00 (Profit)
    print("\n--- Executing a Sell Trade---")
    firm_ledger.execute_trade(ticker="AAPL", side="SELL", quantity=50.0, price_per_unit=195.00)

    # --- Final Ledger Review ---
    print("\n" + "="*50)
    print(f"[{firm_ledger.firm_name}] FINAL LEDGER STATE")
    print("="*50)
    print(f"Cash Balance: ${firm_ledger.get_cash_balance():,.2f}")

    print("\nAsset Holdings:")
    holdings = firm_ledger.get_holdings()
    if holdings:
        for ticker, qty in holdings.items():
            print(f"  {ticker}: {qty:,.2f} units")
    else:
        print("  (No assets currently held)")

    print("\nTransaction Log (First 3 Entries):")
    for i, tx in enumerate(firm_ledger.get_transaction_log()):
        if i >=3:
            break
        print(f" [{i+1}] {tx['type']} {tx.get('ticker', '')}  | Amount: ${tx['amount']:,.2f}  | Cash After: ${tx['cash_after_tx']:,.2f}")

    print(f"Total Transactions: {len(firm_ledger.get_transaction_log())}")

[Airan Finstock Private Limited] Smart Ledger Initialized.
[SUCCESS] Deposited $500,000.00. New Balance: $500,000.00

--- Trading Activity ---
[SUCCESS] BOUGHT 100.00 of AAPL for $18,000.00.
[SUCCESS] BOUGHT 50.00 of GOOGL for $75,000.00.

--- Attempting to Buy Failed Trade ----
[SUCCESS] BOUGHT 1,000.00 of MSFT for $400,000.00.

--- Executing a Sell Trade---
[SUCCESS] SOLD 50.00 of AAPL for $9,750.00.

[Airan Finstock Private Limited] FINAL LEDGER STATE
Cash Balance: $16,750.00

Asset Holdings:
  AAPL: 50.00 units
  GOOGL: 50.00 units
  MSFT: 1,000.00 units

Transaction Log (First 3 Entries):
 [1] DEPOSIT None  | Amount: $500,000.00  | Cash After: $500,000.00
 [2] BUY AAPL  | Amount: $18,000.00  | Cash After: $482,000.00
 [3] BUY GOOGL  | Amount: $75,000.00  | Cash After: $407,000.00
Total Transactions: 5


In [ ]:
""